In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Data Processing
import pandas as pd
import numpy as np

#Data Visulaisation
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.style.use("dark_background")

In [ ]:
data= pd.read_csv('../input/indian-startup-funding/startup_funding.csv')
data.head()

In [ ]:
# Checking Data Attributes

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
#checking for null values
null_check=data.isnull().sum().reset_index()
null_check.columns=['Columns','Number of Nulls']
null_check['% Of Nulls']=((null_check['Number of Nulls']/len(data))*100).round(2)
null_check

In [ ]:
# 86% of the values in Remarks columns are null.Dropping the columns that are not required

In [ ]:
data.drop(['Sr No','Remarks'],axis=1,inplace=True)

In [ ]:
data['Startup Name']=data['Startup Name'].str.replace('0','')
data['Industry Vertical']=data['Industry Vertical'].str.replace('0','')
data['City  Location']=data['City  Location'].str.replace('0','')
data['Investors Name']=data['Investors Name'].str.replace('0','')
#data['Startup Name', 'Industry Vertical','City  Location', 'Investors Name']].str.replace('0','')

In [ ]:
# Filling the data with mode and mean values

In [ ]:
data['SubVertical'].fillna(data['SubVertical'].mode()[0],inplace=True)
data['Industry Vertical'].fillna(data['Industry Vertical'].mode()[0],inplace=True)

In [ ]:
#Converting 'Amount in USD' to integer

In [ ]:
data["Amount in USD"]=data["Amount in USD"].str.replace(",","")
data["Amount in USD"]=data["Amount in USD"].str.replace("undisclosed","")
data["Amount in USD"]=data["Amount in USD"].str.replace("Undisclosed","")
data["Amount in USD"]=data["Amount in USD"].str.replace("0N/A","")
data["Amount in USD"]=data["Amount in USD"].str.replace("unknown","")
data["Amount in USD"]=data["Amount in USD"].str.replace("+","")

In [ ]:
filter1=data[data['Investors Name']=='Atul Jalan']
filter2=data[data['Industry Vertical'].str.contains("xc2")]

data.drop(filter1.index, inplace = True)
data.drop(filter2.index, inplace = True)

In [ ]:
data["Amount in USD"]=pd.to_numeric(data["Amount in USD"])

In [ ]:
data['Amount in USD'].fillna(data['Amount in USD'].mean(),inplace=True)

In [ ]:
#Converting date to datetime and extracting month and year values

In [ ]:
data["Date dd/mm/yyyy"]=data["Date dd/mm/yyyy"].str.replace("05/072018","05/07/2018")
data["Date dd/mm/yyyy"]=data["Date dd/mm/yyyy"].str.replace("01/07/015","01/07/2015")
data["Date dd/mm/yyyy"]=data["Date dd/mm/yyyy"].str.replace("010/7/2015","01/07/2015")
data["Date dd/mm/yyyy"]=data["Date dd/mm/yyyy"].str.replace("12/05.2015","12/05/2015")
data["Date dd/mm/yyyy"]=data["Date dd/mm/yyyy"].str.replace("13/04.2015","13/04/2015")
data["Date dd/mm/yyyy"]=data["Date dd/mm/yyyy"].str.replace("15/01.2015","15/01/2015")
data["Date dd/mm/yyyy"]=data["Date dd/mm/yyyy"].str.replace("22/01//2015","22/01/2015")

In [ ]:
data['Date dd/mm/yyyy']=pd.to_datetime(data['Date dd/mm/yyyy'])

In [ ]:
data['Year']=data['Date dd/mm/yyyy'].dt.year
data['Month']=data['Date dd/mm/yyyy'].dt.month

In [ ]:
plt.figure(figsize=(10,4))
g1=sns.heatmap(data=data.isnull(),yticklabels=False,cmap='BuGn')
g1.set(title="Distribution of null values in the data")

In [ ]:
#Who are the Top 10 investors in Indian ecossystem?

In [ ]:
Top_investors=pd.DataFrame(data.groupby(['Investors Name']).sum()["Amount in USD"]).sort_values(by="Amount in USD",ascending=False)

In [ ]:
plt.figure(figsize=(15,10))
color1=['#9E6737','#AA743A','#B6823D','#C09141','#C9A046','#D1AF4B','#D7C051','#DDD059','#E1E162','#E4F26E']
sns.set_palette(sns.color_palette(color1))
g2=sns.barplot(data=(Top_investors[0:10]).reset_index(),x="Amount in USD",y='Investors Name')
for a in g2.patches:
    g2.annotate('{:.0f}'.format((a.get_width()/100000000).round(2)),
              ((a.get_width(),a.get_y()+a.get_height()/2)),
               ha="left", va="top",
               xytext=(0,9),
               textcoords="offset points")
    
g2.set(title="Top 10 Ivestors by amount Invested")

In [ ]:
#In which city are the highest number of startups located

In [ ]:
data_cities=data.dropna(how="any",axis=0,subset=['City  Location'])
data_cities.shape

In [ ]:
data_cities_sum=pd.DataFrame(data_cities.groupby(['City  Location']).sum()["Amount in USD"])
data_cities_sum['% Investements']=((data_cities_sum['Amount in USD']/data_cities_sum['Amount in USD'].sum())*100).round(2)
data_cities_sum=data_cities_sum.reset_index().sort_values(by='% Investements',ascending=False)

In [ ]:
Top_10 = data_cities_sum[['City  Location','% Investements']][0:10]
Top_10.loc[len(Top_10.index)] = ['Others', 100-(data_cities_sum['% Investements'][0:10]).sum()] 
Top_10=Top_10.sort_values(by='% Investements',ascending=False)

In [ ]:
plt.figure(figsize=(6,6))
plt.title("Distribution of Investments by City")
colors1 = ['#191970','#001CF0','#0038E2','#0055D4','#0071C6','#008DB8','#00AAAA','#00C69C','#00E28E','#00FF80','#4EF4D1']
colors=['#234753','#215861','#206A6B','#247D73','#318F77','#46A178','#60B276','#7EC372','#A0D36E','#C6E16A','#EFEE69']
plt.axis("equal")
g3=plt.pie(Top_10['% Investements'],labels=Top_10['City  Location'],colors=colors,autopct='%0.0f%%',
           shadow=True,explode=[0, 0, 0, 0, 0.1, 0.2,0.3,0.3,0.4,0.5,0.6])

plt.show()

In [ ]:
#Bengaluru sees the highest investmensts followed by Mumbai & New Delhi

In [ ]:
#Which industry verticals are most favored by the investors for funding?

In [ ]:
Inv_verticals_Top_10=pd.DataFrame(data.groupby(['Industry Vertical']).sum()["Amount in USD"]).reset_index().sort_values(by='Amount in USD',ascending=False)[0:10]

In [ ]:
plt.figure(figsize=(15,7))
color2=['#4BC49E','#5CC28C','#6EBF7A','#7FBB69','#91B65B','#A1B14F','#B1AA48','#C0A345','#CD9C46','#D8944C']
sns.set_palette(sns.color_palette(color2))
g4=sns.barplot(data=Inv_verticals_Top_10,x='Industry Vertical',y='Amount in USD')
g4.set(title="Top Industry Verticals secuing Funding")

for a in g4.patches:
    g4.annotate('{:.0f}'.format((a.get_height()/100000000).round(2)),
                ((a.get_x()+a.get_width()/2),a.get_height()),
                ha="center",va="center",
                xytext=(0,9),
                textcoords= 'offset points'
               )

In [ ]:
#Consumer Internet and eCommerce are the industries investors are most interested in

In [ ]:
#How is the funding ecosystem evolving with time ?

In [ ]:
data['YearMonth']=data['Year']*100+data['Month']

In [ ]:
data_time=data[['Date dd/mm/yyyy','YearMonth','City  Location','Amount in USD']]
data_time=data_time.dropna(how="any",axis=0,subset=['City  Location'])

In [ ]:
data_time=pd.DataFrame(data_time[['YearMonth','City  Location']].value_counts()).reset_index().sort_values(by=['City  Location','YearMonth'],ascending=True)

In [ ]:
data_time=data_time[data_time['City  Location'].isin(['Bengaluru','Mumbai','New Delhi','Gurgaon','Noida'])]

In [ ]:
data_time.columns=['YearMonth','City  Location','Count']

In [ ]:
color3=['#50F9F1','#6AE5A8','#96C96A','#B7AA47','#C78845']
sns.set_palette(sns.color_palette(color3))
g5=sns.catplot(data=data_time,x='YearMonth',y='Count',hue='City  Location',kind="point",height=5, aspect=3.0)
g5.set(title='Evolution of funding investments in Top cities')
plt.xticks(rotation=90)
plt.show()